In [ ]:
import pandas as pd
import plotly.express as px

# 1. Data discovery
First step was the manual inspection of the data. This showed the following:
- columns in both data sources seemingly have missing values (empty).
- empty values in *hotel_information* can potentially be in two forms: empty strings or value of `-1`, specifically for the `review_score`.
    - empty values can be treated as missing for now, pending further potential investigation with experts/stakeholders
    - value of `-1` in the `review_score` could also have different interpretation than a missing value, since both are present in the column. Again, this is pending further investigation with experts/stakeholders, but a reasonable assumptions could be:
        - hotel has not been reviewed **yet**. This is lightly supported by the fact that the `stars` column is empty for all of these hotels, although the opposite is not true, i.e. all empty `stars` values do not correspond to `review_score == -1`.
        - review score is not present at the source (therefore not exactly missing but not present) - this is less likely. 
- empty values in *pricing_data* show a clear pattern: when the hotel is sold out (`is_sold_out == True`), all columns but `our_hotel_id`, `arrival_date`, `lead_time` are empty.
- empty values in this case can have a different interpretation than simply missing values. The data is scraped/sourced from rental sites and when there is no availability, it is to be expected that information pertaining to an "available room" would not be present. Thus, no available room, means no corresponding information is available. It is crucial to keep this in mind, so an not to treat these cases as simple missing values and proceed to, for example, impute them.

For the programmatic approach, we start off with basic information of the dataset, to get a sense of the state of the data, distributions, null values, data types etc.

In [ ]:
price_df = pd.read_csv('./data/pricing_data_(5).csv')
info_df = pd.read_csv('./data/hotels_information_(6).csv')

In [ ]:
info_df.info()

In [ ]:
price_df.info()

In [ ]:
info_df.describe()

In [ ]:
price_df.describe()

Quick inspection with `info()` and `describe()` is useful to gain a rough overview of the state of the data. The most important point to keep in mind for later steps (exploration, visualization and modeling) is that the data types seem consistent with what we would expect.


On the negative side though, we can see that NaN values are not detected properly, so below we investigate a bit further.

In [ ]:
price_df.isnull().sum()

In [ ]:
info_df.isnull().sum()

At this stage, looking at the number of nulls in each column and data source, we can explore their patterns and also verify some of our observations from the manual data inspection:
1. Do all 2624 null occurrences observed in the pricing data coincide:
2. `is_sold_out == True` always has null values on the expected columns and vice versa?
3. Is there anything noteworthy to observe with the extra null values in `price_value_ref` column?

In [ ]:
price_df.loc[price_df['is_sold_out']==True].isnull().sum()

In [ ]:
price_null_inv_df = price_df.loc[(price_df['is_sold_out']==False) & (price_df['price_value_ref']).isnull()]

print("[>] Hotels and listed arrival dates with null price_value_ref that are not sold out")
print(price_null_inv_df['our_hotel_id'].unique())
print(price_null_inv_df['arrival_date'].unique())

Answering our previous questions:
- 1 & 2: All 2624 instances coincide and correspond to the sold out hotels.
- 3: There is a pattern in the missing price data, only specific hotels and on the same 6, concecutive dates. There could potentially be a bussiness-related or data ingestion-related reason for this.

Next step would be to join the two sources, using the `our_hotel_id` key, to move forward with our exploration. Pre-joining checks, to manage expectations of the joined dataframe:
- How many unique ids are present in the two data sources?
- Are they the same between the two dataframes?

In [ ]:
unique_info_ids = info_df['our_hotel_id'].unique()
unique_price_ids = price_df['our_hotel_id'].unique()
print(f"[>] Unique hotel ids in price df: {len(unique_price_ids)}")
print(f"[>] Unique hotel ids in info df: {len(unique_info_ids)}")
all_info_in_price = all(i in unique_price_ids for i in unique_info_ids)
all_price_in_info = all(i in unique_info_ids for i in unique_price_ids)
print(f"[>] All ids in info are in price: {all_info_in_price}")
print(f"[>] All ids in price are in info: {all_price_in_info}")

In [ ]:
data = price_df.merge(info_df, on="our_hotel_id", how="left", suffixes=["_price", "_info"])

In [ ]:
data

With the combined data, we can replace the missing values, to facilitate our exploration.
- For numeric values, NaN values replaced with `-1`: although `-1` could have a particular meaning for `review_score`, for the purposes of our exploration, we can use it as a placeholder for missing values. For the purposes of this exploration, we uniformly apply this replacement, knowing the difference in meaning for sold out and not sold out properties. In case the missing `price_value_ref` values need to be imputed, the `is_sold_out` column will allow for clear separation of the two aforementioned cases. This, however, falls out of the scope of this exploration. 
- For string values, NaN values are replaced with `<N/A>`: a "not applicable" is preferred for the string columns, since they correspond to the pricing dataset's columns and all fall under the case of sold out hotels. Therefore "not applicable" is better suited than "unknown" or "missing".

Main reason for this handling is to be able to cast these columns to their appropriate type and still allow for easy filtering of these cases.

In [ ]:
value_mapping = {
    'price_value_ref': -1,
    'price_value_non_ref': -1,
    'max_persons': -1,
    'review_score': -1,
    'stars': -1,
    'room_count': -1,
    'room_name': '<UNK>',
    'meal_type_included': '<UNK>',
}
data.fillna(value=value_mapping)

# 2. Markets comparison 

## 2.1 Detecting hotel markets
Latitude and longitude being part of the dataset is a good indicator on the possible differentiator, therefore the data is displayed against a map. We easily observe two groups, dutch and belgian hotels. With some manual inspection of latitude values on the map, we can easily differentiate them. For ease of further exploration, a new column, `location`, is created.  

In [ ]:
fig = px.scatter_mapbox(
    data, 
    lat='latitude', 
    lon='longitude',
    hover_name='name',
    hover_data=["review_score", "stars", "room_count"],
    zoom=5
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})


In [ ]:
data.loc[data['latitude'] > 51, 'location'] = 'NL'
data.loc[data['latitude'] < 51, 'location'] = 'BE'
print(f"[>] Number of dutch hotels: {info_df[info_df['latitude'] > 51].shape[0]}")
print(f"[>] Number of belgian hotels: {info_df[info_df['latitude'] < 51].shape[0]}")

### 2.2.1 Hotel availability

Comparing the availability of the two groups, we see different patterns for the two countries:
- Belgium: availability is high during January with a clear downwards trend towards April. This could be expected, since the Spring months are usually preferred by visitors. Additionally, the peak period of December and Christmas is not included, that would justify a dip in availability despite the worse weather conditions. Slight periodicity could be present on an x-day or week basis, so further exploration might be meaningful in that direction.
- The Netherlands: availability follows a similar downwards trend from January to April. However, overall availability is lower compared to Belgium, although more dutch hotels are present in the dataset. Additionally, there is clear weekly periodicity in the data. Although the availability during the week becomes less and less, availability during the weekends spikes (quick manual look-up for the dates shows us weekdays from weekends). This is an interesting pattern, worth exploring further along with experts/stakeholders.

In [ ]:
arr_grp = data.groupby(
    ['arrival_date', 'location', 'is_sold_out']
).size().reset_index(name='counts')

In [ ]:
arr_grp

In [ ]:
px.bar(
    arr_grp.loc[arr_grp['is_sold_out']==True],
    x='arrival_date',
    y='counts',
    color='location',
    barmode='group',
    title='Available hotels per arrival date by group'
)

### 2.2.2 Pricing patterns
We are already aware of the issues with missing values, therefore for the purpose of this exploration all entries with missing price data are excluded. This includes the non sold out instances as well, but for our purposes, it should not pose a problem.

Both refundable and non refundable prices follow similar patterns for the two countries. Mean and minimum prices are also following a similar pattern. Clear differences can only be observed on the maximum price between the two countries. That obviously concerns the top priced hotel in the area, so this difference might not pose significant interest. Perhaps only the (still isolated) fact in the Netherlands, even the top hotel seems to follow the weekly periodicity observed also in the availability, namely weekends are more expensive but more availability is also observed. 

In [ ]:
prc_grp = data.loc[(data['price_value_ref'] > -1) & (data['price_value_non_ref'] > -1)].groupby(
    ['arrival_date', 'location']
).agg(
    min_ref_price=pd.NamedAgg(column='price_value_ref', aggfunc='min'),
    avg_ref_price=pd.NamedAgg(column='price_value_ref', aggfunc='mean'),
    max_ref_price=pd.NamedAgg(column='price_value_ref', aggfunc='max'),
    min_non_ref_price=pd.NamedAgg(column='price_value_non_ref', aggfunc='min'),
    avg_non_ref_price=pd.NamedAgg(column='price_value_non_ref', aggfunc='mean'),
    max_non_ref_price=pd.NamedAgg(column='price_value_non_ref', aggfunc='max'),
)

In [ ]:
prc_grp

In [ ]:
px.line(
    prc_grp.reset_index(),
    x='arrival_date',
    y=['min_ref_price', 'avg_ref_price', 'max_ref_price'],
    facet_row='location'
)

In [ ]:
px.line(
    prc_grp.reset_index(),
    x='arrival_date',
    y=['min_non_ref_price', 'avg_non_ref_price', 'max_non_ref_price'],
    facet_row='location'
)

In [ ]:
px.box(
    data.loc[(data['price_value_ref'] > -1) & (data['review_score'] > -1)],
    'review_score',
    'price_value_ref',
    color='location'
)

### 2.2.3 Stars and review scores distribution

Initial observations show that dutch hotels reach lower and higher review scores than belgian ones. This is generally consistent between stars and reviews.

In [ ]:
rev_grp = data.loc[(data['review_score'] > -1) & (data['stars'] > -1)].groupby(
    ['location']
).agg(
    min_review_score=pd.NamedAgg(column='review_score', aggfunc='min'),
    avg_review_score=pd.NamedAgg(column='review_score', aggfunc='mean'),
    max_review_score=pd.NamedAgg(column='review_score', aggfunc='max'),
    min_stars=pd.NamedAgg(column='stars', aggfunc='min'),
    avg_stars=pd.NamedAgg(column='stars', aggfunc='mean'),
    max_stars=pd.NamedAgg(column='stars', aggfunc='max'),
)

In [ ]:
rev_grp

Hotels' ratings are also briefly explored against prices, to investigate potential interesting patterns, especially if they go against simple logic like "more expensive is more highly rated".

There are a couple of potentially interesting distributions and comparisons between the two groups. For example the Netherlands seems to have a few more highly reviewed hotels (>9) and that Belgium has somewhat less spread of prices per review score.

In [ ]:
px.box(
    data.loc[(data['price_value_ref'] > -1) & (data['review_score'] > -1)],
    'review_score',
    'price_value_ref',
    color='location'
)

In [ ]:
px.box(
    data.loc[(data['price_value_non_ref'] > -1) & (data['review_score'] > -1)],
    'review_score',
    'price_value_non_ref',
    color='location'
)

### 2.2.4 Potential explorations

Points of interest would be:
- The effect of including a meal to the rating, price and availability of a hotel. This can help businesses decide on the benefits and detriments of including this option.
- The relation between hotel size (number of rooms) and price, rating and availability. These insights could start a discussion around questions of the nature "are big hotels charging more or less", "is size of a hotel indicative of its rating" etc.
- More high-level insights, maybe concerning also client behavioural patterns. For example: are hotels that are frequently not available more or less popular? Does consistenly lower availability make a hotel saught after or does it discourage clients? Such questions could be potentially pursued by utilizing a combination of price trends, ratings and availability.